In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import os
import glob
import codecs
import csv
import prepare

In [2]:
#display all columns 
pd.set_option("display.max_columns", None)

In [3]:
#pulling in main csv
df = pd.read_csv('ATPTotal.csv')
df.head(1)

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2.0,NaN,Kei Nishikori,R,178.0,JPN,29.004791,106421,4.0,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,6-4 3-6 6-2,3,F,124.0,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0


In [4]:
#df[df.winner_id == 104053]

In [5]:
#### changed NANS to 101 so they would get filtered out later on and not corrupt a player's highest rank over 100

df['winner_rank'] = df['winner_rank'].fillna(101)
df['loser_rank'] = df['loser_rank'].fillna(101)

In [6]:
df.winner_rank.value_counts()

101.0     34158
1.0        3040
2.0        2877
3.0        2753
4.0        2435
          ...  
605.0         1
1053.0        1
1083.0        1
1051.0        1
1026.0        1
Name: winner_rank, Length: 1218, dtype: int64

In [7]:
# df[df.winner_name == 'Andy Roddick'].winner_rank.sort_values()

In [8]:
df = df.set_index('tourney_date')
# convert index to datetime object and sort in ascending order
df.index = pd.to_datetime(df.index, format = '%Y%m%d')
df = df.sort_index(ascending = True)
df = df['1999-01-01':'2020-01-01']
# drop all walkovers (no useful stats) and best of 1 matches (extremely rare format)
df = df.drop(df[df.score == 'W/O'].index)
df = df.drop(df[df.best_of == 1].index)

In [9]:
df.head(1)

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
tourney_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-11,108,1999-338,Sydney,Hard,32,A,16,103720,NaN,NaN,Lleyton Hewitt,R,180.0,AUS,17.878166,102158,2.0,NaN,Patrick Rafter,R,185.0,AUS,26.036961,7-6(1) 6-1,3,R32,84.0,2.0,0.0,73.0,51.0,32.0,14.0,10.0,4.0,6.0,0.0,5.0,59.0,36.0,24.0,8.0,9.0,3.0,7.0,104.0,456.0,4.0,3315.0


In [10]:
#pulling 
#Create dataframe for Player Data and label columns
PlayerColumns = ['PlayerID', 'Player_Name', 'Age', 'Height', 'MaxRank', 'Hand', 'Country']

PlayerData = pd.DataFrame(columns = PlayerColumns)
PlayerData.set_index('PlayerID')


#Add data to the PlayerData dataframe
for index, row in df.iterrows():
    #player was the winner
    p_id = int(row.winner_id)
    
    #Check value
    if(p_id not in PlayerData.index):
        PlayerData.loc[p_id] = [p_id, row.winner_name, row.winner_age, row.winner_ht, row.winner_rank, row.winner_hand, row.winner_ioc]

    #Update rank with best rank value
    if(row.winner_rank < PlayerData.loc[p_id].MaxRank and not np.isnan(PlayerData.loc[p_id].MaxRank)):
        PlayerData.loc[p_id] = [p_id, row.winner_name, row.winner_age, row.winner_ht, row.winner_rank, row.winner_hand, row.winner_ioc]

    #Player was loser
    p2_id = int(row.loser_id)
    #Check for loser
    if(p2_id not in PlayerData.index):
        PlayerData.loc[p2_id] = [p2_id, row.loser_name, row.loser_age, row.loser_ht, row.loser_rank, row.loser_hand, row.loser_ioc]

    #Update rank with best rank value
    if(row.loser_rank < PlayerData.loc[p2_id].MaxRank and not np.isnan(PlayerData.loc[p2_id].MaxRank)):
        PlayerData.loc[p2_id] = [p2_id, row.loser_name, row.loser_age, row.loser_ht, row.loser_rank, row.loser_hand, row.loser_ioc]


In [11]:
#looking at main player details
PlayerData

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country
103720,103720,Lleyton Hewitt,20.752909,180.0,1.0,R,AUS
102158,102158,Patrick Rafter,26.477755,185.0,2.0,R,AUS
103017,103017,Nicolas Kiefer,22.535250,183.0,4.0,R,GER
210013,210013,Martin Damm Sr,28.227242,188.0,67.0,R,CZE
103264,103264,Mariano Puerta,26.904860,180.0,9.0,L,ARG
...,...,...,...,...,...,...,...
144984,144984,Artem Dubrivnyy,20.700890,NaN,424.0,R,RUS
200553,200553,Alen Avidzba,19.635866,NaN,498.0,R,RUS
208230,208230,Alibek Kachmazov,17.158111,NaN,1491.0,R,RUS
127339,127339,Borna Gojo,21.722108,NaN,280.0,R,CRO


In [12]:
# Win Count DF
win_count = df.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(int).reset_index()
win_count = win_count.rename(columns={'winner_name': 'win_count', 'winner_id' : 'PlayerID'})
win_count

,PlayerID,win_count
0,100644,149
1,100754,3
2,100943,1
3,101086,8
4,101150,39
...,...,...
1444,208028,1
1445,208029,2
1446,208055,1
1447,209188,1


In [13]:
# Loss Count DF
loss_count = df.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(int).reset_index()
loss_count = loss_count.rename(columns={'loser_name': 'lose_count', 'loser_id' : 'PlayerID'})
loss_count

,PlayerID,lose_count
0,100644,71
1,100754,4
2,100943,2
3,100997,2
4,101086,19
...,...,...
2133,208597,1
2134,208937,1
2135,209226,1
2136,209238,1


In [14]:
### merge win_loss counts

from functools import reduce

data_frames = [win_count, loss_count]

win_loss = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)

win_loss = win_loss.dropna()
win_loss['match_count'] = win_loss['win_count'] + win_loss['lose_count']
win_loss['win%'] = round((win_loss['win_count'] / win_loss['match_count']),2)

win_loss

,PlayerID,win_count,lose_count,match_count,win%
0,100644,149,71.0,220.0,0.68
1,100754,3,4.0,7.0,0.43
2,100943,1,2.0,3.0,0.33
3,101086,8,19.0,27.0,0.30
4,101150,39,55.0,94.0,0.41
...,...,...,...,...,...
1440,206173,9,5.0,14.0,0.64
1441,207139,2,2.0,4.0,0.50
1445,208029,2,1.0,3.0,0.67
1446,208055,1,1.0,2.0,0.50


In [15]:
df.head(1)

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
tourney_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-11,108,1999-338,Sydney,Hard,32,A,16,103720,NaN,NaN,Lleyton Hewitt,R,180.0,AUS,17.878166,102158,2.0,NaN,Patrick Rafter,R,185.0,AUS,26.036961,7-6(1) 6-1,3,R32,84.0,2.0,0.0,73.0,51.0,32.0,14.0,10.0,4.0,6.0,0.0,5.0,59.0,36.0,24.0,8.0,9.0,3.0,7.0,104.0,456.0,4.0,3315.0


In [16]:
loser_second_serve = df.groupby('loser_id')[['l_2ndWon']].sum().reset_index()
loser_second_serve = loser_second_serve.rename(columns = {'loser_id' : 'PlayerID'})
#loser_second_serve
#loser_first_serve_percent = loser_first_serve_percent.rename(columns={'loser_first_serve_percent': 'l_first_serve', 'loser_id' : 'PlayerID'})

winner_second_serve = df.groupby('winner_id')[['w_2ndWon']].sum().reset_index()
winner_second_serve = winner_second_serve.rename(columns = {'winner_id' : 'PlayerID'})
winner_second_serve


from functools import reduce

data_frames = [winner_second_serve, loser_second_serve, win_loss]

second_serve = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)

second_serve.fillna(0)
second_serve['second_serve_count'] = second_serve['w_2ndWon'] + second_serve['l_2ndWon']
second_serve['second_serve_match%'] = round(second_serve['second_serve_count'] / second_serve['match_count'],3)
columns_drop =['win_count', 'lose_count', 'match_count', 'win%', 'w_2ndWon', 'l_2ndWon', ]
second_serve = second_serve.drop(columns_drop, 1)

second_serve

<ipython-input-16-8f7707a624d2>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  second_serve = second_serve.drop(columns_drop, 1)


,PlayerID,second_serve_count,second_serve_match%
0,100644,3217.0,14.623
1,100754,0.0,0.000
2,100943,0.0,0.000
3,101086,353.0,13.074
4,101150,1917.0,20.394
...,...,...,...
1444,208028,NaN,NaN
1445,208029,18.0,6.000
1446,208055,0.0,0.000
1447,209188,NaN,NaN


In [17]:
#First Serve win percentage added new column
df['w_first_serve'] = df['w_1stIn'] / df['w_svpt']
df['l_first_serve'] = df['l_1stIn'] / df['l_svpt']

#First Serve Win
df['w_first_serve_win'] = df['w_1stWon'] / df['w_1stIn']
df['l_first_serve_win'] = df['l_1stWon'] / df['l_1stIn']



In [18]:


loser_first_serve_percent = df.groupby('loser_id')[['l_first_serve']].mean().reset_index()
loser_first_serve_percent = loser_first_serve_percent.rename(columns={'loser_first_serve_percent': 'l_first_serve', 'loser_id' : 'PlayerID'})

winner_first_serve_percent = df.groupby('winner_id')[['w_first_serve']].mean().reset_index()
winner_first_serve_percent = winner_first_serve_percent.rename(columns={'winner_first_serve_percent': 'w_first_serve', 'winner_id' : 'PlayerID'})


loser_first_serve_win_percent = df.groupby('loser_id')[['l_first_serve_win']].mean().reset_index()
loser_first_serve_win_percent = loser_first_serve_win_percent.rename(columns={'loser_first_serve_win_percent': 'l_first_serve_win', 'loser_id' : 'PlayerID'})

winner_first_serve_win_percent = df.groupby('winner_id')[['w_first_serve_win']].mean().reset_index()
winner_first_serve_win_percent = winner_first_serve_win_percent.rename(columns={'winner_first_serve_win_percent': 'w_first_serve_win', 'winner_id' : 'PlayerID'})


### merge serve stats

from functools import reduce

data_frames = [loser_first_serve_percent, winner_first_serve_percent, loser_first_serve_win_percent, winner_first_serve_win_percent]

serve_pts = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
serve_pts

,PlayerID,l_first_serve,w_first_serve,l_first_serve_win,w_first_serve_win
0,100644,0.617291,0.645815,0.654301,0.778817
1,100754,NaN,NaN,NaN,NaN
2,100943,NaN,NaN,NaN,NaN
3,100997,NaN,NaN,NaN,NaN
4,101086,0.644105,0.676382,0.591278,0.702205
...,...,...,...,...,...
2133,208597,NaN,NaN,NaN,NaN
2134,208937,NaN,NaN,NaN,NaN
2135,209226,NaN,NaN,NaN,NaN
2136,209238,NaN,NaN,NaN,NaN


In [19]:
# from functools import reduce

# data_frames = [PlayerData, win_loss, serve_pts]

# PD_merge = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
#                                             how='left'), data_frames)
# PD_merge

In [20]:
### Breakpoints
#df.head(1)
df['w_break_points_won'] = df['l_bpFaced']- df['l_bpSaved']
df['l_break_points_won'] = df['w_bpFaced']- df['w_bpSaved']

loser_breakpoints_won = df.groupby('loser_id')[['l_break_points_won']].agg('sum').reset_index()
loser_breakpoints_won = loser_breakpoints_won.rename(columns={'loser_id' : 'PlayerID'})

winner_breakpoints_won = df.groupby('winner_id')[['w_break_points_won']].agg('sum').reset_index()
winner_breakpoints_won = winner_breakpoints_won.rename(columns={'winner_id' : 'PlayerID'})



data_frames = [loser_breakpoints_won, winner_breakpoints_won]

breakpoints = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
breakpoints['breakpoint_count'] = breakpoints['l_break_points_won'] + breakpoints['w_break_points_won']
breakpoints

data_frames = [breakpoints, win_loss]
breakpoints = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
breakpoints['breakpoints_per_game'] = breakpoints['breakpoint_count'] / breakpoints['match_count']
columns_drop =['win_count', 'lose_count', 'match_count', 'win%']
breakpoints = breakpoints.drop(columns_drop, 1)
breakpoints


<ipython-input-20-ef02aba40de2>:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  breakpoints = breakpoints.drop(columns_drop, 1)


,PlayerID,l_break_points_won,w_break_points_won,breakpoint_count,breakpoints_per_game
0,100644,138.0,559.0,697.0,3.168182
1,100754,0.0,0.0,0.0,0.000000
2,100943,0.0,0.0,0.0,0.000000
3,100997,0.0,NaN,NaN,NaN
4,101086,42.0,32.0,74.0,2.740741
...,...,...,...,...,...
2133,208597,0.0,NaN,NaN,NaN
2134,208937,0.0,NaN,NaN,NaN
2135,209226,0.0,NaN,NaN,NaN
2136,209238,0.0,NaN,NaN,NaN


In [21]:
loser_breakpoints_won = df.groupby('loser_id')[['l_break_points_won']].agg('sum').reset_index()
loser_breakpoints_won = loser_breakpoints_won.rename(columns={'loser_id' : 'PlayerID'})
loser_breakpoints_won

,PlayerID,l_break_points_won
0,100644,138.0
1,100754,0.0
2,100943,0.0
3,100997,0.0
4,101086,42.0
...,...,...
2133,208597,0.0
2134,208937,0.0
2135,209226,0.0
2136,209238,0.0


In [22]:
loss_aces = df.groupby('loser_id')[['l_ace']].agg('sum').reset_index()
loss_aces = loss_aces.rename(columns={'loser_id' : 'PlayerID', 'lose_count': 'ace_count_lost'})
loss_aces

,PlayerID,l_ace
0,100644,511.0
1,100754,0.0
2,100943,0.0
3,100997,0.0
4,101086,45.0
...,...,...
2133,208597,0.0
2134,208937,0.0
2135,209226,0.0
2136,209238,0.0


In [23]:
loss_aces[loss_aces['PlayerID'] == 103333 ]

,PlayerID,l_ace
524,103333,4813.0


In [24]:
win_aces = df.groupby('winner_id')[['w_ace']].agg('sum').reset_index()
win_aces = win_aces.rename(columns={'winner_id' : 'PlayerID', 'win_count':'ace_count_win'})
win_aces

,PlayerID,w_ace
0,100644,1227.0
1,100754,0.0
2,100943,0.0
3,101086,16.0
4,101150,268.0
...,...,...
1444,208028,1.0
1445,208029,5.0
1446,208055,0.0
1447,209188,0.0


In [25]:
win_aces[win_aces['PlayerID'] == 103333 ]

,PlayerID,w_ace
403,103333,5263.0


In [26]:
### ACES
loss_aces = df.groupby('loser_id')[['l_ace']].agg('sum').reset_index()
loss_aces = loss_aces.rename(columns={'loser_id' : 'PlayerID', 'lose_count': 'ace_count_lost'})

win_aces = df.groupby('winner_id')[['w_ace']].agg('sum').reset_index()
win_aces = win_aces.rename(columns={'winner_id' : 'PlayerID', 'win_count':'ace_count_win'})

data_frames = [loss_aces, win_aces]

aces = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
aces['ace_count'] = aces['l_ace'] + aces['w_ace']

data_frames = [aces, win_loss]
aces = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
aces['aces_per_game'] = round((aces['ace_count'] / aces['match_count']),3)
columns_drop =['win_count', 'lose_count', 'match_count', 'win%']
aces = aces.drop(columns_drop, 1)
aces

<ipython-input-26-b0986138a851>:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  aces = aces.drop(columns_drop, 1)


,PlayerID,l_ace,w_ace,ace_count,aces_per_game
0,100644,511.0,1227.0,1738.0,7.900
1,100754,0.0,0.0,0.0,0.000
2,100943,0.0,0.0,0.0,0.000
3,100997,0.0,NaN,NaN,NaN
4,101086,45.0,16.0,61.0,2.259
...,...,...,...,...,...
2133,208597,0.0,NaN,NaN,NaN
2134,208937,0.0,NaN,NaN,NaN
2135,209226,0.0,NaN,NaN,NaN
2136,209238,0.0,NaN,NaN,NaN


In [27]:
aces[aces['PlayerID'] == 103333 ].isnull().sum()

PlayerID         0
l_ace            0
w_ace            0
ace_count        0
aces_per_game    0
dtype: int64

In [28]:
#### Count Matches WON & LOSS with opponent being in the top 30 ###
#9744 rows of loser is in top 30

match_30 = df[df.loser_rank <=30]
match_30

win_30 = match_30.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(int).reset_index()
win_30 = win_30.rename(columns={'winner_name': 'win_count_30', 'winner_id' : 'PlayerID'})

matchl_30 = df[df.winner_rank <=30]
loss_30 = matchl_30.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(int).reset_index()
loss_30 = loss_30.rename(columns={'loser_name': 'loss_count_30', 'loser_id' : 'PlayerID'})

#9744 rows of loser is in top 30
#df[df.loser_rank <=30]

In [29]:
#### Count Matches WON & LOSS with opponent being in the top 100 ###
#32632 rows of loser is in top 100

match_100 = df[df.loser_rank <=100]
match_100

win_100 = match_100.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(int).reset_index()
win_100 = win_100.rename(columns={'winner_name': 'win_count_100', 'winner_id' : 'PlayerID'})

matchl_100 = df[df.winner_rank <=100]
loss_100 = matchl_100.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(int).reset_index()
loss_100 = loss_100.rename(columns={'loser_name': 'loss_count_100', 'loser_id' : 'PlayerID'})


### we now have win_100 and loss_100

In [30]:
from functools import reduce

data_frames = [win_30, loss_30, win_100, loss_100]

top_matches = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)

## nulls will change to zero as player did not lose to either top 30 or top 100
## loss_counts 30 = 14 & loss_counts 100 = 2
top_matches = top_matches.fillna(0)

top_matches['total_top30_matches'] = top_matches['win_count_30'] + top_matches['loss_count_30']
top_matches['total_top100_matches'] = top_matches['win_count_100'] + top_matches['loss_count_100']

top_matches['top_30_win%'] = top_matches['win_count_30'] / top_matches['total_top30_matches']
top_matches['top_100_win%'] = top_matches['win_count_100'] / top_matches['total_top100_matches']


top_matches

,PlayerID,win_count_30,loss_count_30,win_count_100,loss_count_100,total_top30_matches,total_top100_matches,top_30_win%,top_100_win%
0,100644,48,46.0,120,67.0,94.0,187.0,0.510638,0.641711
1,101150,3,18.0,18,44.0,21.0,62.0,0.142857,0.290323
2,101185,4,4.0,6,16.0,8.0,22.0,0.500000,0.272727
3,101235,1,7.0,5,11.0,8.0,16.0,0.125000,0.312500
4,101320,4,12.0,31,36.0,16.0,67.0,0.250000,0.462687
...,...,...,...,...,...,...,...,...,...
523,200282,6,12.0,37,22.0,18.0,59.0,0.333333,0.627119
524,200325,1,0.0,1,2.0,1.0,3.0,1.000000,0.333333
525,200615,1,2.0,6,11.0,3.0,17.0,0.333333,0.352941
526,206173,2,1.0,7,5.0,3.0,12.0,0.666667,0.583333


In [31]:
df.head(1)

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,w_first_serve,l_first_serve,w_first_serve_win,l_first_serve_win,w_break_points_won,l_break_points_won
tourney_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-11,108,1999-338,Sydney,Hard,32,A,16,103720,NaN,NaN,Lleyton Hewitt,R,180.0,AUS,17.878166,102158,2.0,NaN,Patrick Rafter,R,185.0,AUS,26.036961,7-6(1) 6-1,3,R32,84.0,2.0,0.0,73.0,51.0,32.0,14.0,10.0,4.0,6.0,0.0,5.0,59.0,36.0,24.0,8.0,9.0,3.0,7.0,104.0,456.0,4.0,3315.0,0.69863,0.610169,0.627451,0.666667,4.0,2.0


In [32]:
df.surface.value_counts()

Hard      24917
Clay      16704
Grass      4927
Carpet     1755
Name: surface, dtype: int64

In [33]:
### creating databases for surface type
hard = df[df.surface == 'Hard']
clay = df[df.surface == 'Clay']
grass = df[df.surface == 'Grass']
carpet = df[df.surface == 'Carpet']

#Hard
hard_win = hard.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(float).reset_index()
hard_win = hard_win.rename(columns={'winner_name': 'hard_surface_win', 'winner_id' : 'PlayerID'})
hard_loss = hard.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(float).reset_index()
hard_loss = hard_loss.rename(columns={'loser_name': 'hard_surface_loss', 'loser_id' : 'PlayerID'})
data_frames = [hard_win, hard_loss]
hard_df = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
#putting losses to 0 count
hard_df = hard_df.fillna(0)
hard_df['hard_surface_match_count'] = hard_df['hard_surface_win'] + hard_df['hard_surface_loss']
hard_df['hard_win%'] = hard_df['hard_surface_win'] / hard_df['hard_surface_match_count']

# #Clay
clay_win = clay.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(float).reset_index()
clay_win = clay_win.rename(columns={'winner_name': 'clay_surface_win', 'winner_id' : 'PlayerID'})
clay_loss = clay.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(float).reset_index()
clay_loss = clay_loss.rename(columns={'loser_name': 'clay_surface_loss', 'loser_id' : 'PlayerID'})
data_frames = [clay_win, clay_loss]
clay_df = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
#putting losses to 0 count
clay_df = clay_df.fillna(0)
clay_df['clay_surface_match_count'] = clay_df['clay_surface_win'] + clay_df['clay_surface_loss']
clay_df['clay_win%'] = clay_df['clay_surface_win'] / clay_df['clay_surface_match_count']


#Grass
grass_win = grass.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(float).reset_index()
grass_win = grass_win.rename(columns={'winner_name': 'grass_surface_win', 'winner_id' : 'PlayerID'})
grass_loss = grass.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(float).reset_index()
grass_loss = grass_loss.rename(columns={'loser_name': 'grass_surface_loss', 'loser_id' : 'PlayerID'})
data_frames = [grass_win, grass_loss]
grass_df = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
#putting losses to 0 count
grass_df = grass_df.fillna(0)
grass_df['grass_surface_match_count'] = grass_df['grass_surface_win'] + grass_df['grass_surface_loss']
grass_df['grass_win%'] = grass_df['grass_surface_win'] / grass_df['grass_surface_match_count']

#Carpet
carpet_win = carpet.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(float).reset_index()
carpet_win = carpet_win.rename(columns={'winner_name': 'carpet_surface_win', 'winner_id' : 'PlayerID'})
carpet_loss = carpet.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(float).reset_index()
carpet_loss = carpet_loss.rename(columns={'loser_name': 'carpet_surface_loss', 'loser_id' : 'PlayerID'})
data_frames = [carpet_win, carpet_loss]
carpet_df = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
#putting losses to 0 count
carpet_df = carpet_df.fillna(0)
carpet_df['carpet_surface_match_count'] = carpet_df['carpet_surface_win'] + carpet_df['carpet_surface_loss']
carpet_df['carpet_win%'] = carpet_df['carpet_surface_win'] / carpet_df['carpet_surface_match_count']

data_frames = [hard_df, clay_df, grass_df, carpet_df]
surface = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)

surface.isnull().sum()


PlayerID                        0
hard_surface_win                0
hard_surface_loss               0
hard_surface_match_count        0
hard_win%                       0
clay_surface_win              432
clay_surface_loss             432
clay_surface_match_count      432
clay_win%                     432
grass_surface_win             670
grass_surface_loss            670
grass_surface_match_count     670
grass_win%                    670
carpet_surface_win            828
carpet_surface_loss           828
carpet_surface_match_count    828
carpet_win%                   828
dtype: int64

In [34]:
print(f'hard_df shape: {hard_df.shape}')
print(f'clay_df shape: {clay_df.shape}')
print(f'grass_df shape: {grass_df.shape}')
print(f'carpet_df shape: {carpet_df.shape}')


hard_df shape: (1132, 5)
clay_df shape: (960, 5)
grass_df shape: (507, 5)
carpet_df shape: (352, 5)


In [35]:
data_frames = [hard_df, clay_df, grass_df, carpet_df]
surface = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)

### in each surface df nulls were only present for players that didn't lose a match on that surface type
### merging all surface dfs presents nulls because their are unequal representation of rows

#surface.isnull().sum()

surface= surface.fillna(0)

In [36]:
## COMBINING ALL DATAFRAMES TOGETHER

from functools import reduce

data_frames = [PlayerData, win_loss, aces, serve_pts, second_serve, breakpoints, top_matches, surface]

PD_merge = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)

PD_merge_predict = PD_merge
PD_merge


,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,l_ace,w_ace,ace_count,aces_per_game,l_first_serve,w_first_serve,l_first_serve_win,w_first_serve_win,second_serve_count,second_serve_match%,l_break_points_won,w_break_points_won,breakpoint_count,breakpoints_per_game,win_count_30,loss_count_30,win_count_100,loss_count_100,total_top30_matches,total_top100_matches,top_30_win%,top_100_win%,hard_surface_win,hard_surface_loss,hard_surface_match_count,hard_win%,clay_surface_win,clay_surface_loss,clay_surface_match_count,clay_win%,grass_surface_win,grass_surface_loss,grass_surface_match_count,grass_win%,carpet_surface_win,carpet_surface_loss,carpet_surface_match_count,carpet_win%
0,103720,Lleyton Hewitt,20.752909,180.0,1.0,R,AUS,458.0,196.0,654.0,0.70,1034.0,2823.0,3857.0,5.898,0.525558,0.539098,0.675974,0.773470,12096.0,18.495,407.0,1947.0,2354.0,3.599388,134.0,110.0,351.0,184.0,244.0,535.0,0.549180,0.656075,265.0,118.0,383.0,0.691906,85.0,46.0,131.0,0.648855,93.0,27.0,120.0,0.775000,15.0,5.0,20.0,0.750000
1,102158,Patrick Rafter,26.477755,185.0,2.0,R,AUS,97.0,44.0,141.0,0.69,292.0,637.0,929.0,6.589,0.637279,0.653328,0.700358,0.795185,1987.0,14.092,61.0,342.0,403.0,2.858156,35.0,25.0,75.0,38.0,60.0,113.0,0.583333,0.663717,47.0,25.0,72.0,0.652778,14.0,11.0,25.0,0.560000,31.0,6.0,37.0,0.837838,5.0,2.0,7.0,0.714286
2,103017,Nicolas Kiefer,22.535250,183.0,4.0,R,GER,217.0,162.0,379.0,0.57,975.0,1514.0,2489.0,6.567,0.514047,0.537990,0.689337,0.792370,6490.0,17.124,240.0,836.0,1076.0,2.839050,70.0,87.0,168.0,148.0,157.0,316.0,0.445860,0.531646,141.0,88.0,229.0,0.615721,37.0,44.0,81.0,0.456790,19.0,14.0,33.0,0.575758,20.0,16.0,36.0,0.555556
3,210013,Martin Damm Sr,28.227242,188.0,67.0,R,CZE,19.0,32.0,51.0,0.37,203.0,141.0,344.0,6.745,0.546553,0.611743,0.693080,0.806999,855.0,16.765,38.0,72.0,110.0,2.156863,4.0,9.0,11.0,25.0,13.0,36.0,0.307692,0.305556,9.0,16.0,25.0,0.360000,4.0,7.0,11.0,0.363636,6.0,7.0,13.0,0.461538,0.0,0.0,0.0,0.000000
4,103264,Mariano Puerta,26.904860,180.0,9.0,L,ARG,76.0,79.0,155.0,0.49,272.0,273.0,545.0,3.516,0.637432,0.680268,0.613399,0.746508,1844.0,11.897,93.0,272.0,365.0,2.354839,11.0,33.0,55.0,71.0,44.0,126.0,0.250000,0.436508,14.0,21.0,35.0,0.400000,61.0,48.0,109.0,0.559633,0.0,0.0,0.0,0.000000,1.0,7.0,8.0,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,144984,Artem Dubrivnyy,20.700890,NaN,424.0,R,RUS,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.632353,NaN,0.465116,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2231,200553,Alen Avidzba,19.635866,NaN,498.0,R,RUS,1.0,1.0,2.0,0.50,1.0,11.0,12.0,6.000,0.492537,0.625000,0.606061,0.777778,25.0,12.500,2.0,6.0,8.0,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,0.500000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2232,208230,Alibek Kachmazov,17.158111,NaN,1491.0,R,RUS,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,0.518072,NaN,0.534884,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2233,127339,Borna Gojo,21.722108,NaN,280.0,R,CRO,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,0.616055,NaN,0.643789,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
PD_merge.ace_count.sort_values()

994     0.0
916     0.0
918     0.0
921     0.0
305     0.0
       ... 
2229    NaN
2230    NaN
2232    NaN
2233    NaN
2234    NaN
Name: ace_count, Length: 2235, dtype: float64

In [38]:
PD_merge.Player_Name

0                Lleyton Hewitt
1                Patrick Rafter
2                Nicolas Kiefer
3                Martin Damm Sr
4                Mariano Puerta
                 ...           
2230            Artem Dubrivnyy
2231               Alen Avidzba
2232           Alibek Kachmazov
2233                 Borna Gojo
2234    Botic Van De Zandschulp
Name: Player_Name, Length: 2235, dtype: object

In [39]:
rank1 = PD_merge[PD_merge['MaxRank'] == 1]
rank1.shape

(13, 49)

In [40]:
PD_merge[PD_merge.Player_Name == 'Andy Roddick']

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,l_ace,w_ace,ace_count,aces_per_game,l_first_serve,w_first_serve,l_first_serve_win,w_first_serve_win,second_serve_count,second_serve_match%,l_break_points_won,w_break_points_won,breakpoint_count,breakpoints_per_game,win_count_30,loss_count_30,win_count_100,loss_count_100,total_top30_matches,total_top100_matches,top_30_win%,top_100_win%,hard_surface_win,hard_surface_loss,hard_surface_match_count,hard_win%,clay_surface_win,clay_surface_loss,clay_surface_match_count,clay_win%,grass_surface_win,grass_surface_loss,grass_surface_match_count,grass_win%,carpet_surface_win,carpet_surface_loss,carpet_surface_match_count,carpet_win%
471,104053,Andy Roddick,21.196441,188.0,1.0,R,USA,430.0,151.0,581.0,0.74,1473.0,4830.0,6303.0,10.849,0.624105,0.654332,0.728889,0.820516,8084.0,13.914,151.0,1250.0,1401.0,2.41136,111.0,83.0,344.0,142.0,194.0,486.0,0.572165,0.707819,303.0,100.0,403.0,0.751861,58.0,31.0,89.0,0.651685,54.0,14.0,68.0,0.794118,15.0,6.0,21.0,0.714286


In [41]:
#PD_merge[PD_merge.Player_Name == 'Andy Roddick'] = 1

In [42]:


#df.replace({0: 10, 1: 100})

In [43]:
#[PD_merge.Player_Name == 'Andy Roddick' 

In [44]:
columns_to_rename = {'l_ace' : 'aces_in_match_lost',
                    'w_ace': 'aces_in_match_won',
                    'l_first_serve': 'first_serve_percentage_match_lost',
                    'w_first_serve': 'first_serve_percentage_match_won',
                    'l_first_serve_win': 'first_serve_won_percentage_match_lost',
                    'w_first_serve_win': 'first_serve_won_percentage_match_won',
                    'l_break_points_won': 'breakpoints_won_match_lost',
                    'w_break_points_won': 'breakpoints_won_match_won'}
PD_merge = PD_merge.rename(columns = columns_to_rename)
PD_merge_predict = PD_merge


In [45]:
PD_merge_predict

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,aces_in_match_lost,aces_in_match_won,ace_count,aces_per_game,first_serve_percentage_match_lost,first_serve_percentage_match_won,first_serve_won_percentage_match_lost,first_serve_won_percentage_match_won,second_serve_count,second_serve_match%,breakpoints_won_match_lost,breakpoints_won_match_won,breakpoint_count,breakpoints_per_game,win_count_30,loss_count_30,win_count_100,loss_count_100,total_top30_matches,total_top100_matches,top_30_win%,top_100_win%,hard_surface_win,hard_surface_loss,hard_surface_match_count,hard_win%,clay_surface_win,clay_surface_loss,clay_surface_match_count,clay_win%,grass_surface_win,grass_surface_loss,grass_surface_match_count,grass_win%,carpet_surface_win,carpet_surface_loss,carpet_surface_match_count,carpet_win%
0,103720,Lleyton Hewitt,20.752909,180.0,1.0,R,AUS,458.0,196.0,654.0,0.70,1034.0,2823.0,3857.0,5.898,0.525558,0.539098,0.675974,0.773470,12096.0,18.495,407.0,1947.0,2354.0,3.599388,134.0,110.0,351.0,184.0,244.0,535.0,0.549180,0.656075,265.0,118.0,383.0,0.691906,85.0,46.0,131.0,0.648855,93.0,27.0,120.0,0.775000,15.0,5.0,20.0,0.750000
1,102158,Patrick Rafter,26.477755,185.0,2.0,R,AUS,97.0,44.0,141.0,0.69,292.0,637.0,929.0,6.589,0.637279,0.653328,0.700358,0.795185,1987.0,14.092,61.0,342.0,403.0,2.858156,35.0,25.0,75.0,38.0,60.0,113.0,0.583333,0.663717,47.0,25.0,72.0,0.652778,14.0,11.0,25.0,0.560000,31.0,6.0,37.0,0.837838,5.0,2.0,7.0,0.714286
2,103017,Nicolas Kiefer,22.535250,183.0,4.0,R,GER,217.0,162.0,379.0,0.57,975.0,1514.0,2489.0,6.567,0.514047,0.537990,0.689337,0.792370,6490.0,17.124,240.0,836.0,1076.0,2.839050,70.0,87.0,168.0,148.0,157.0,316.0,0.445860,0.531646,141.0,88.0,229.0,0.615721,37.0,44.0,81.0,0.456790,19.0,14.0,33.0,0.575758,20.0,16.0,36.0,0.555556
3,210013,Martin Damm Sr,28.227242,188.0,67.0,R,CZE,19.0,32.0,51.0,0.37,203.0,141.0,344.0,6.745,0.546553,0.611743,0.693080,0.806999,855.0,16.765,38.0,72.0,110.0,2.156863,4.0,9.0,11.0,25.0,13.0,36.0,0.307692,0.305556,9.0,16.0,25.0,0.360000,4.0,7.0,11.0,0.363636,6.0,7.0,13.0,0.461538,0.0,0.0,0.0,0.000000
4,103264,Mariano Puerta,26.904860,180.0,9.0,L,ARG,76.0,79.0,155.0,0.49,272.0,273.0,545.0,3.516,0.637432,0.680268,0.613399,0.746508,1844.0,11.897,93.0,272.0,365.0,2.354839,11.0,33.0,55.0,71.0,44.0,126.0,0.250000,0.436508,14.0,21.0,35.0,0.400000,61.0,48.0,109.0,0.559633,0.0,0.0,0.0,0.000000,1.0,7.0,8.0,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,144984,Artem Dubrivnyy,20.700890,NaN,424.0,R,RUS,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.632353,NaN,0.465116,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2231,200553,Alen Avidzba,19.635866,NaN,498.0,R,RUS,1.0,1.0,2.0,0.50,1.0,11.0,12.0,6.000,0.492537,0.625000,0.606061,0.777778,25.0,12.500,2.0,6.0,8.0,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,0.500000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2232,208230,Alibek Kachmazov,17.158111,NaN,1491.0,R,RUS,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,0.518072,NaN,0.534884,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2233,127339,Borna Gojo,21.722108,NaN,280.0,R,CRO,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,0.616055,NaN,0.643789,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# UPDATED 03/06

In [46]:
### filtering for players who only play 50 matches and their max rank was over 100

PD_merge = PD_merge[PD_merge.match_count > 50]
PD_merge = PD_merge[PD_merge.MaxRank < 100]
PD_merge

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,aces_in_match_lost,aces_in_match_won,ace_count,aces_per_game,first_serve_percentage_match_lost,first_serve_percentage_match_won,first_serve_won_percentage_match_lost,first_serve_won_percentage_match_won,second_serve_count,second_serve_match%,breakpoints_won_match_lost,breakpoints_won_match_won,breakpoint_count,breakpoints_per_game,win_count_30,loss_count_30,win_count_100,loss_count_100,total_top30_matches,total_top100_matches,top_30_win%,top_100_win%,hard_surface_win,hard_surface_loss,hard_surface_match_count,hard_win%,clay_surface_win,clay_surface_loss,clay_surface_match_count,clay_win%,grass_surface_win,grass_surface_loss,grass_surface_match_count,grass_win%,carpet_surface_win,carpet_surface_loss,carpet_surface_match_count,carpet_win%
0,103720,Lleyton Hewitt,20.752909,180.0,1.0,R,AUS,458.0,196.0,654.0,0.70,1034.0,2823.0,3857.0,5.898,0.525558,0.539098,0.675974,0.773470,12096.0,18.495,407.0,1947.0,2354.0,3.599388,134.0,110.0,351.0,184.0,244.0,535.0,0.549180,0.656075,265.0,118.0,383.0,0.691906,85.0,46.0,131.0,0.648855,93.0,27.0,120.0,0.775000,15.0,5.0,20.0,0.750000
1,102158,Patrick Rafter,26.477755,185.0,2.0,R,AUS,97.0,44.0,141.0,0.69,292.0,637.0,929.0,6.589,0.637279,0.653328,0.700358,0.795185,1987.0,14.092,61.0,342.0,403.0,2.858156,35.0,25.0,75.0,38.0,60.0,113.0,0.583333,0.663717,47.0,25.0,72.0,0.652778,14.0,11.0,25.0,0.560000,31.0,6.0,37.0,0.837838,5.0,2.0,7.0,0.714286
2,103017,Nicolas Kiefer,22.535250,183.0,4.0,R,GER,217.0,162.0,379.0,0.57,975.0,1514.0,2489.0,6.567,0.514047,0.537990,0.689337,0.792370,6490.0,17.124,240.0,836.0,1076.0,2.839050,70.0,87.0,168.0,148.0,157.0,316.0,0.445860,0.531646,141.0,88.0,229.0,0.615721,37.0,44.0,81.0,0.456790,19.0,14.0,33.0,0.575758,20.0,16.0,36.0,0.555556
3,210013,Martin Damm Sr,28.227242,188.0,67.0,R,CZE,19.0,32.0,51.0,0.37,203.0,141.0,344.0,6.745,0.546553,0.611743,0.693080,0.806999,855.0,16.765,38.0,72.0,110.0,2.156863,4.0,9.0,11.0,25.0,13.0,36.0,0.307692,0.305556,9.0,16.0,25.0,0.360000,4.0,7.0,11.0,0.363636,6.0,7.0,13.0,0.461538,0.0,0.0,0.0,0.000000
4,103264,Mariano Puerta,26.904860,180.0,9.0,L,ARG,76.0,79.0,155.0,0.49,272.0,273.0,545.0,3.516,0.637432,0.680268,0.613399,0.746508,1844.0,11.897,93.0,272.0,365.0,2.354839,11.0,33.0,55.0,71.0,44.0,126.0,0.250000,0.436508,14.0,21.0,35.0,0.400000,61.0,48.0,109.0,0.559633,0.0,0.0,0.0,0.000000,1.0,7.0,8.0,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,200282,Alex De Minaur,20.711841,183.0,18.0,R,AUS,45.0,27.0,72.0,0.62,100.0,219.0,319.0,4.431,0.615800,0.628429,0.638539,0.775507,1141.0,15.847,39.0,177.0,216.0,3.000000,6.0,12.0,37.0,22.0,18.0,59.0,0.333333,0.627119,40.0,16.0,56.0,0.714286,2.0,7.0,9.0,0.222222,3.0,4.0,7.0,0.428571,0.0,0.0,0.0,0.000000
2065,126774,Stefanos Tsitsipas,20.788501,193.0,6.0,R,GRE,62.0,37.0,99.0,0.63,307.0,483.0,790.0,7.980,0.584801,0.624753,0.735252,0.784889,1813.0,18.313,55.0,188.0,243.0,2.454545,26.0,18.0,51.0,35.0,44.0,86.0,0.590909,0.593023,39.0,25.0,64.0,0.609375,18.0,8.0,26.0,0.692308,5.0,4.0,9.0,0.555556,0.0,0.0,0.0,0.000000
2078,105815,Tennys Sandgren,27.482546,188.0,41.0,R,USA,25.0,30.0,55.0,0.45,222.0,255.0,477.0,8.673,0.600247,0.633775,0.692401,0.799943,871.0,15.836,44.0,95.0,139.0,2.527273,6.0,7.0,18.0,24.0,13.0,42.0,0.461538,0.428571,13.0,13.0,26.0,0.500000,7.0,12.0,19.0,0.368421,5.0,5.0,10.0,0.500000,0.0,0.0,0.0,0.000000
2081,126610,Matteo Berrettini,23.581109,196.0,8.0,R,ITA,29.0,25.0,54.0,0.54,168.0,263.0,431.0,7.981,0.610410,0.628175,0.702783,0.822480,883.0,16.352,22.0,93.0,115.0,2.129630,6.0,10.0,21.0,21.0,16.0,42.0,0.375000,0.500000,9.0,13.0,22.0,0.409091,15.0,9.0,24.0,0.625000,5.0,3.0,8.0,0.625000,0.0,0.0,0.0,0.000000


In [47]:


# ## PlayerData = over 50 games, top 100 rank, 1999-2019
#PD_merge.to_csv('PlayerData.csv')

# ## note to upload this CSV 

In [48]:
PD_merge[PD_merge.Player_Name == 'Nick Kyrgios']

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,aces_in_match_lost,aces_in_match_won,ace_count,aces_per_game,first_serve_percentage_match_lost,first_serve_percentage_match_won,first_serve_won_percentage_match_lost,first_serve_won_percentage_match_won,second_serve_count,second_serve_match%,breakpoints_won_match_lost,breakpoints_won_match_won,breakpoint_count,breakpoints_per_game,win_count_30,loss_count_30,win_count_100,loss_count_100,total_top30_matches,total_top100_matches,top_30_win%,top_100_win%,hard_surface_win,hard_surface_loss,hard_surface_match_count,hard_win%,clay_surface_win,clay_surface_loss,clay_surface_match_count,clay_win%,grass_surface_win,grass_surface_loss,grass_surface_match_count,grass_win%,carpet_surface_win,carpet_surface_loss,carpet_surface_match_count,carpet_win%
1754,106401,Nick Kyrgios,21.724846,193.0,13.0,R,AUS,87.0,58.0,145.0,0.6,711.0,1396.0,2107.0,14.531,0.63524,0.646478,0.69963,0.820426,2308.0,15.917,67.0,253.0,320.0,2.206897,25.0,34.0,73.0,49.0,59.0,122.0,0.423729,0.598361,57.0,37.0,94.0,0.606383,18.0,14.0,32.0,0.5625,12.0,7.0,19.0,0.631579,0.0,0.0,0.0,0.0


In [49]:
PD_merge[PD_merge.MaxRank < 5].head(50)

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,aces_in_match_lost,aces_in_match_won,ace_count,aces_per_game,first_serve_percentage_match_lost,first_serve_percentage_match_won,first_serve_won_percentage_match_lost,first_serve_won_percentage_match_won,second_serve_count,second_serve_match%,breakpoints_won_match_lost,breakpoints_won_match_won,breakpoint_count,breakpoints_per_game,win_count_30,loss_count_30,win_count_100,loss_count_100,total_top30_matches,total_top100_matches,top_30_win%,top_100_win%,hard_surface_win,hard_surface_loss,hard_surface_match_count,hard_win%,clay_surface_win,clay_surface_loss,clay_surface_match_count,clay_win%,grass_surface_win,grass_surface_loss,grass_surface_match_count,grass_win%,carpet_surface_win,carpet_surface_loss,carpet_surface_match_count,carpet_win%
0,103720,Lleyton Hewitt,20.752909,180.0,1.0,R,AUS,458.0,196.0,654.0,0.70,1034.0,2823.0,3857.0,5.898,0.525558,0.539098,0.675974,0.773470,12096.0,18.495,407.0,1947.0,2354.0,3.599388,134.0,110.0,351.0,184.0,244.0,535.0,0.549180,0.656075,265.0,118.0,383.0,0.691906,85.0,46.0,131.0,0.648855,93.0,27.0,120.0,0.775000,15.0,5.0,20.0,0.750000
1,102158,Patrick Rafter,26.477755,185.0,2.0,R,AUS,97.0,44.0,141.0,0.69,292.0,637.0,929.0,6.589,0.637279,0.653328,0.700358,0.795185,1987.0,14.092,61.0,342.0,403.0,2.858156,35.0,25.0,75.0,38.0,60.0,113.0,0.583333,0.663717,47.0,25.0,72.0,0.652778,14.0,11.0,25.0,0.560000,31.0,6.0,37.0,0.837838,5.0,2.0,7.0,0.714286
2,103017,Nicolas Kiefer,22.535250,183.0,4.0,R,GER,217.0,162.0,379.0,0.57,975.0,1514.0,2489.0,6.567,0.514047,0.537990,0.689337,0.792370,6490.0,17.124,240.0,836.0,1076.0,2.839050,70.0,87.0,168.0,148.0,157.0,316.0,0.445860,0.531646,141.0,88.0,229.0,0.615721,37.0,44.0,81.0,0.456790,19.0,14.0,33.0,0.575758,20.0,16.0,36.0,0.555556
6,101774,Todd Martin,29.278576,198.0,4.0,R,USA,100.0,68.0,168.0,0.60,624.0,1182.0,1806.0,10.750,0.601599,0.627814,0.721192,0.792471,2879.0,17.137,113.0,354.0,467.0,2.779762,28.0,41.0,83.0,64.0,69.0,147.0,0.405797,0.564626,67.0,40.0,107.0,0.626168,16.0,18.0,34.0,0.470588,17.0,9.0,26.0,0.653846,0.0,0.0,0.0,0.000000
8,102845,Carlos Moya,22.546201,190.0,1.0,R,ESP,297.0,167.0,464.0,0.64,837.0,1714.0,2551.0,5.498,0.576697,0.587575,0.674769,0.769395,7932.0,17.095,286.0,1050.0,1336.0,2.879310,80.0,80.0,221.0,149.0,160.0,370.0,0.500000,0.597297,102.0,79.0,181.0,0.563536,181.0,76.0,257.0,0.704280,7.0,7.0,14.0,0.500000,7.0,5.0,12.0,0.583333
14,101990,Richard Krajicek,27.310062,196.0,4.0,R,NED,69.0,34.0,103.0,0.67,443.0,1008.0,1451.0,14.087,0.586349,0.578956,0.748001,0.858824,1733.0,16.825,45.0,242.0,287.0,2.786408,22.0,15.0,53.0,30.0,37.0,83.0,0.594595,0.638554,36.0,18.0,54.0,0.666667,12.0,8.0,20.0,0.600000,12.0,5.0,17.0,0.705882,9.0,3.0,12.0,0.750000
16,102856,Gustavo Kuerten,24.347707,190.0,1.0,R,BRA,202.0,106.0,308.0,0.66,727.0,1709.0,2436.0,7.909,0.552564,0.556285,0.718379,0.796136,5281.0,17.146,155.0,743.0,898.0,2.915584,52.0,53.0,145.0,89.0,105.0,234.0,0.495238,0.619658,72.0,51.0,123.0,0.585366,108.0,40.0,148.0,0.729730,7.0,4.0,11.0,0.636364,15.0,11.0,26.0,0.576923
26,102374,Alex Corretja,24.752909,180.0,3.0,R,ESP,135.0,90.0,225.0,0.60,368.0,616.0,984.0,4.373,0.528088,0.534388,0.642742,0.755873,4228.0,18.791,167.0,544.0,711.0,3.160000,42.0,39.0,110.0,78.0,81.0,188.0,0.518519,0.585106,44.0,39.0,83.0,0.530120,88.0,42.0,130.0,0.676923,1.0,2.0,3.0,0.333333,2.0,7.0,9.0,0.222222
34,103163,Tommy Haas,24.109514,188.0,2.0,R,GER,393.0,232.0,625.0,0.63,1147.0,2928.0,4075.0,6.520,0.574516,0.593934,0.679912,0.796813,9897.0,15.835,324.0,1342.0,1666.0,2.665600,105.0,120.0,314.0,204.0,225.0,518.0,0.466667,0.606178,246.0,131.0,377.0,0.652520,97.0,67.0,164.0,0.591463,32.0,23.0,55.0,0.581818,18.0,11.0,29.0,0.620690
40,103498,Marat Safin,20.835044,193.0,1.0,R,RUS,319.0,180.0,499.0,0.64,1299.0,2866.0,4165.0,8.347,0.552617,0.556719,0.707889,0.788261,9081.0,18.198,284.0,1109.0,1393.0,2.791583,97.0,80.0,252.0,161.0,177.0,413.0,0.548023,0.610169,158.0,95.0,253.0,0.624506,94.0,55.

In [50]:
907/1109

0.8178539224526601

In [51]:
PD_merge[PD_merge.Player_Name == 'Roger Federer']

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,aces_in_match_lost,aces_in_match_won,ace_count,aces_per_game,first_serve_percentage_match_lost,first_serve_percentage_match_won,first_serve_won_percentage_match_lost,first_serve_won_percentage_match_won,second_serve_count,second_serve_match%,breakpoints_won_match_lost,breakpoints_won_match_won,breakpoint_count,breakpoints_per_game,win_count_30,loss_count_30,win_count_100,loss_count_100,total_top30_matches,total_top100_matches,top_30_win%,top_100_win%,hard_surface_win,hard_surface_loss,hard_surface_match_count,hard_win%,clay_surface_win,clay_surface_loss,clay_surface_match_count,clay_win%,grass_surface_win,grass_surface_loss,grass_surface_match_count,grass_win%,carpet_surface_win,carpet_surface_loss,carpet_surface_match_count,carpet_win%
133,103819,Roger Federer,22.485969,185.0,1.0,R,SUI,907.0,212.0,1119.0,0.81,1541.0,6829.0,8370.0,7.48,0.597457,0.629201,0.701538,0.796579,18099.0,16.174,391.0,3238.0,3629.0,3.243074,407.0,146.0,786.0,200.0,553.0,986.0,0.735986,0.79716,552.0,120.0,672.0,0.821429,189.0,57.0,246.0,0.768293,132.0,19.0,151.0,0.874172,34.0,16.0,50.0,0.68


In [52]:
PD_merge_predict

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,aces_in_match_lost,aces_in_match_won,ace_count,aces_per_game,first_serve_percentage_match_lost,first_serve_percentage_match_won,first_serve_won_percentage_match_lost,first_serve_won_percentage_match_won,second_serve_count,second_serve_match%,breakpoints_won_match_lost,breakpoints_won_match_won,breakpoint_count,breakpoints_per_game,win_count_30,loss_count_30,win_count_100,loss_count_100,total_top30_matches,total_top100_matches,top_30_win%,top_100_win%,hard_surface_win,hard_surface_loss,hard_surface_match_count,hard_win%,clay_surface_win,clay_surface_loss,clay_surface_match_count,clay_win%,grass_surface_win,grass_surface_loss,grass_surface_match_count,grass_win%,carpet_surface_win,carpet_surface_loss,carpet_surface_match_count,carpet_win%
0,103720,Lleyton Hewitt,20.752909,180.0,1.0,R,AUS,458.0,196.0,654.0,0.70,1034.0,2823.0,3857.0,5.898,0.525558,0.539098,0.675974,0.773470,12096.0,18.495,407.0,1947.0,2354.0,3.599388,134.0,110.0,351.0,184.0,244.0,535.0,0.549180,0.656075,265.0,118.0,383.0,0.691906,85.0,46.0,131.0,0.648855,93.0,27.0,120.0,0.775000,15.0,5.0,20.0,0.750000
1,102158,Patrick Rafter,26.477755,185.0,2.0,R,AUS,97.0,44.0,141.0,0.69,292.0,637.0,929.0,6.589,0.637279,0.653328,0.700358,0.795185,1987.0,14.092,61.0,342.0,403.0,2.858156,35.0,25.0,75.0,38.0,60.0,113.0,0.583333,0.663717,47.0,25.0,72.0,0.652778,14.0,11.0,25.0,0.560000,31.0,6.0,37.0,0.837838,5.0,2.0,7.0,0.714286
2,103017,Nicolas Kiefer,22.535250,183.0,4.0,R,GER,217.0,162.0,379.0,0.57,975.0,1514.0,2489.0,6.567,0.514047,0.537990,0.689337,0.792370,6490.0,17.124,240.0,836.0,1076.0,2.839050,70.0,87.0,168.0,148.0,157.0,316.0,0.445860,0.531646,141.0,88.0,229.0,0.615721,37.0,44.0,81.0,0.456790,19.0,14.0,33.0,0.575758,20.0,16.0,36.0,0.555556
3,210013,Martin Damm Sr,28.227242,188.0,67.0,R,CZE,19.0,32.0,51.0,0.37,203.0,141.0,344.0,6.745,0.546553,0.611743,0.693080,0.806999,855.0,16.765,38.0,72.0,110.0,2.156863,4.0,9.0,11.0,25.0,13.0,36.0,0.307692,0.305556,9.0,16.0,25.0,0.360000,4.0,7.0,11.0,0.363636,6.0,7.0,13.0,0.461538,0.0,0.0,0.0,0.000000
4,103264,Mariano Puerta,26.904860,180.0,9.0,L,ARG,76.0,79.0,155.0,0.49,272.0,273.0,545.0,3.516,0.637432,0.680268,0.613399,0.746508,1844.0,11.897,93.0,272.0,365.0,2.354839,11.0,33.0,55.0,71.0,44.0,126.0,0.250000,0.436508,14.0,21.0,35.0,0.400000,61.0,48.0,109.0,0.559633,0.0,0.0,0.0,0.000000,1.0,7.0,8.0,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,144984,Artem Dubrivnyy,20.700890,NaN,424.0,R,RUS,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.632353,NaN,0.465116,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2231,200553,Alen Avidzba,19.635866,NaN,498.0,R,RUS,1.0,1.0,2.0,0.50,1.0,11.0,12.0,6.000,0.492537,0.625000,0.606061,0.777778,25.0,12.500,2.0,6.0,8.0,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,0.500000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2232,208230,Alibek Kachmazov,17.158111,NaN,1491.0,R,RUS,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,0.518072,NaN,0.534884,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2233,127339,Borna Gojo,21.722108,NaN,280.0,R,CRO,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,0.616055,NaN,0.643789,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
### same
PD_merge_predict.shape
columns_drop = ['win_count', 'lose_count', 'match_count', 'aces_in_match_lost', 'aces_in_match_won', 'first_serve_percentage_match_lost',
              'first_serve_percentage_match_won', 'first_serve_won_percentage_match_lost', 'first_serve_won_percentage_match_won', 'breakpoints_won_match_lost',
              'breakpoints_won_match_won', 'win_count_30', 'loss_count_30', 'win_count_100', 'loss_count_100', 'total_top30_matches',
              'total_top100_matches', 'hard_surface_win', 'hard_surface_loss', 'hard_surface_match_count', 
              'clay_surface_win', 'clay_surface_loss', 'clay_surface_match_count',
              'grass_surface_win', 'grass_surface_loss', 'grass_surface_match_count',
              'carpet_surface_win', 'carpet_surface_loss', 'carpet_surface_match_count', 'Player_Name',
               'Height', ]

PD_merge_predict = PD_merge_predict.drop(columns_drop, 1)
PD_merge_predict
# aces = aces.drop(columns_drop, 1)

<ipython-input-53-bcd550d22e47>:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  PD_merge_predict = PD_merge_predict.drop(columns_drop, 1)


,PlayerID,Age,MaxRank,Hand,Country,win%,ace_count,aces_per_game,second_serve_count,second_serve_match%,breakpoint_count,breakpoints_per_game,top_30_win%,top_100_win%,hard_win%,clay_win%,grass_win%,carpet_win%
0,103720,20.752909,1.0,R,AUS,0.70,3857.0,5.898,12096.0,18.495,2354.0,3.599388,0.549180,0.656075,0.691906,0.648855,0.775000,0.750000
1,102158,26.477755,2.0,R,AUS,0.69,929.0,6.589,1987.0,14.092,403.0,2.858156,0.583333,0.663717,0.652778,0.560000,0.837838,0.714286
2,103017,22.535250,4.0,R,GER,0.57,2489.0,6.567,6490.0,17.124,1076.0,2.839050,0.445860,0.531646,0.615721,0.456790,0.575758,0.555556
3,210013,28.227242,67.0,R,CZE,0.37,344.0,6.745,855.0,16.765,110.0,2.156863,0.307692,0.305556,0.360000,0.363636,0.461538,0.000000
4,103264,26.904860,9.0,L,ARG,0.49,545.0,3.516,1844.0,11.897,365.0,2.354839,0.250000,0.436508,0.400000,0.559633,0.000000,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,144984,20.700890,424.0,R,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2231,200553,19.635866,498.0,R,RUS,0.50,12.0,6.000,25.0,12.500,8.0,4.000000,NaN,NaN,0.500000,0.000000,0.000000,0.000000
2232,208230,17.158111,1491.0,R,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2233,127339,21.722108,280.0,R,CRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
pd30 = pd.read_csv('PlayerData30.csv')
pd30

,Unnamed: 0,PlayerID,Player_Name,Height(cm),Matches Played,Overall Win%,Top 100 Win %,Top 30 Win %,First Serve %,First Serve Win %,BPSave %,BPConv %,Aces/Match,DF/Match,Future Top 30
0,103720,103720,Lleyton Hewitt,180.0,30,0.700000,0.689655,0.500000,0.614167,0.723759,0.642105,0.450000,3.633333,2.166667,True
1,103017,103017,Nicolas Kiefer,183.0,30,0.633333,0.583333,0.571429,0.512586,0.756438,0.631336,0.421053,6.333333,4.800000,True
2,103264,103264,Mariano Puerta,180.0,30,0.333333,0.259259,0.090909,0.655638,0.652213,0.633094,0.423913,3.866667,2.166667,True
3,102845,102845,Carlos Moya,190.0,30,0.633333,0.545455,0.454545,0.537364,0.683812,0.648464,0.405229,4.266667,3.666667,True
4,102563,102563,Thomas Johansson,180.0,30,0.533333,0.428571,0.375000,0.509013,0.754888,0.625571,0.410959,7.666667,3.233333,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,126774,126774,Stefanos Tsitsipas,193.0,30,0.466667,0.444444,0.454545,0.572467,0.768238,0.600000,0.388889,7.500000,2.466667,True
243,126610,126610,Matteo Berrettini,196.0,30,0.500000,0.409091,0.166667,0.606633,0.763902,0.588235,0.345946,7.366667,2.300000,True
244,111815,111815,Cameron Norrie,188.0,30,0.466667,0.333333,0.272727,0.627597,0.699960,0.642857,0.392405,5.033333,4.100000,False
245,132283,132283,Lorenzo Sonego,191.0,30,0.533333,0.500000,0.333333,0.645355,0.733773,0.624413,0.336898,7.600000,1.733333,False


In [55]:
pd30


columns_to_rename = {'Overall Win%' : 'Overall_Win%_30matches',
                      'Top 100 Win %': 'Top_100_Win%_30matches',
                      'winner_entry': 'Player1_entry',
                      'Top 30 Win %': 'Top_30_Win%_30matches',
                      'First Serve %': 'First_Serve%_30matches',
                      'First Serve Win %': 'First_Serve_Win%_30matches',
                      'BPSave %': 'BPSave%_30matches',
                      'BPConv %': 'BPConv%_30matches',
                      'Aces/Match': 'Aces/Match_30matches',
                      'DF/Match': 'DF/Match_30matches'}
pd30 = pd30.rename(columns = columns_to_rename)
pd30

,Unnamed: 0,PlayerID,Player_Name,Height(cm),Matches Played,Overall_Win%_30matches,Top_100_Win%_30matches,Top_30_Win%_30matches,First_Serve%_30matches,First_Serve_Win%_30matches,BPSave%_30matches,BPConv%_30matches,Aces/Match_30matches,DF/Match_30matches,Future Top 30
0,103720,103720,Lleyton Hewitt,180.0,30,0.700000,0.689655,0.500000,0.614167,0.723759,0.642105,0.450000,3.633333,2.166667,True
1,103017,103017,Nicolas Kiefer,183.0,30,0.633333,0.583333,0.571429,0.512586,0.756438,0.631336,0.421053,6.333333,4.800000,True
2,103264,103264,Mariano Puerta,180.0,30,0.333333,0.259259,0.090909,0.655638,0.652213,0.633094,0.423913,3.866667,2.166667,True
3,102845,102845,Carlos Moya,190.0,30,0.633333,0.545455,0.454545,0.537364,0.683812,0.648464,0.405229,4.266667,3.666667,True
4,102563,102563,Thomas Johansson,180.0,30,0.533333,0.428571,0.375000,0.509013,0.754888,0.625571,0.410959,7.666667,3.233333,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,126774,126774,Stefanos Tsitsipas,193.0,30,0.466667,0.444444,0.454545,0.572467,0.768238,0.600000,0.388889,7.500000,2.466667,True
243,126610,126610,Matteo Berrettini,196.0,30,0.500000,0.409091,0.166667,0.606633,0.763902,0.588235,0.345946,7.366667,2.300000,True
244,111815,111815,Cameron Norrie,188.0,30,0.466667,0.333333,0.272727,0.627597,0.699960,0.642857,0.392405,5.033333,4.100000,False
245,132283,132283,Lorenzo Sonego,191.0,30,0.533333,0.500000,0.333333,0.645355,0.733773,0.624413,0.336898,7.600000,1.733333,False


In [56]:
data_frames = [PD_merge_predict, pd30]

PD_merge_30 = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='inner'), data_frames)


PD_merge_30


,PlayerID,Age,MaxRank,Hand,Country,win%,ace_count,aces_per_game,second_serve_count,second_serve_match%,breakpoint_count,breakpoints_per_game,top_30_win%,top_100_win%,hard_win%,clay_win%,grass_win%,carpet_win%,Unnamed: 0,Player_Name,Height(cm),Matches Played,Overall_Win%_30matches,Top_100_Win%_30matches,Top_30_Win%_30matches,First_Serve%_30matches,First_Serve_Win%_30matches,BPSave%_30matches,BPConv%_30matches,Aces/Match_30matches,DF/Match_30matches,Future Top 30
0,103720,20.752909,1.0,R,AUS,0.70,3857.0,5.898,12096.0,18.495,2354.0,3.599388,0.549180,0.656075,0.691906,0.648855,0.775000,0.750000,103720,Lleyton Hewitt,180.0,30,0.700000,0.689655,0.500000,0.614167,0.723759,0.642105,0.450000,3.633333,2.166667,True
1,103017,22.535250,4.0,R,GER,0.57,2489.0,6.567,6490.0,17.124,1076.0,2.839050,0.445860,0.531646,0.615721,0.456790,0.575758,0.555556,103017,Nicolas Kiefer,183.0,30,0.633333,0.583333,0.571429,0.512586,0.756438,0.631336,0.421053,6.333333,4.800000,True
2,103264,26.904860,9.0,L,ARG,0.49,545.0,3.516,1844.0,11.897,365.0,2.354839,0.250000,0.436508,0.400000,0.559633,0.000000,0.125000,103264,Mariano Puerta,180.0,30,0.333333,0.259259,0.090909,0.655638,0.652213,0.633094,0.423913,3.866667,2.166667,True
3,102845,22.546201,1.0,R,ESP,0.64,2551.0,5.498,7932.0,17.095,1336.0,2.879310,0.500000,0.597297,0.563536,0.704280,0.500000,0.583333,102845,Carlos Moya,190.0,30,0.633333,0.545455,0.454545,0.537364,0.683812,0.648464,0.405229,4.266667,3.666667,True
4,102563,27.214237,7.0,R,SWE,0.54,2881.0,7.981,6187.0,17.139,879.0,2.434903,0.308333,0.489933,0.550505,0.369231,0.666667,0.613636,102563,Thomas Johansson,180.0,30,0.533333,0.428571,0.375000,0.509013,0.754888,0.625571,0.410959,7.666667,3.233333,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,126774,20.788501,6.0,R,GRE,0.63,790.0,7.980,1813.0,18.313,243.0,2.454545,0.590909,0.593023,0.609375,0.692308,0.555556,0.000000,126774,Stefanos Tsitsipas,193.0,30,0.466667,0.444444,0.454545,0.572467,0.768238,0.600000,0.388889,7.500000,2.466667,True
243,126610,23.581109,8.0,R,ITA,0.54,431.0,7.981,883.0,16.352,115.0,2.129630,0.375000,0.500000,0.409091,0.625000,0.625000,0.000000,126610,Matteo Berrettini,196.0,30,0.500000,0.409091,0.166667,0.606633,0.763902,0.588235,0.345946,7.366667,2.300000,True
244,111815,23.663244,45.0,L,GBR,0.49,298.0,5.418,770.0,14.000,141.0,2.563636,0.214286,0.414634,0.516129,0.500000,0.416667,0.000000,111815,Cameron Norrie,188.0,30,0.466667,0.333333,0.272727,0.627597,0.699960,0.642857,0.392405,5.033333,4.100000,False
245,132283,24.139630,46.0,R,ITA,0.48,247.0,7.485,552.0,16.727,64.0,1.939394,0.285714,0.461538,0.363636,0.466667,0.714286,0.000000,132283,Lorenzo Sonego,191.0,30,0.533333,0.500000,0.333333,0.645355,0.733773,0.624413,0.336898,7.600000,1.733333,False


In [57]:
PD_merge_30.isnull().sum()

PlayerID                      0
Age                           0
MaxRank                       0
Hand                          0
Country                       0
win%                          0
ace_count                     0
aces_per_game                 0
second_serve_count            0
second_serve_match%           0
breakpoint_count              0
breakpoints_per_game          0
top_30_win%                   7
top_100_win%                  7
hard_win%                     0
clay_win%                     0
grass_win%                    0
carpet_win%                   0
Unnamed: 0                    0
Player_Name                   0
Height(cm)                    0
Matches Played                0
Overall_Win%_30matches        0
Top_100_Win%_30matches        0
Top_30_Win%_30matches         0
First_Serve%_30matches        0
First_Serve_Win%_30matches    0
BPSave%_30matches             0
BPConv%_30matches             0
Aces/Match_30matches          0
DF/Match_30matches            0
Future T

In [58]:
PD_merge_30 = PD_merge_30.dropna()
PD_merge_30

,PlayerID,Age,MaxRank,Hand,Country,win%,ace_count,aces_per_game,second_serve_count,second_serve_match%,breakpoint_count,breakpoints_per_game,top_30_win%,top_100_win%,hard_win%,clay_win%,grass_win%,carpet_win%,Unnamed: 0,Player_Name,Height(cm),Matches Played,Overall_Win%_30matches,Top_100_Win%_30matches,Top_30_Win%_30matches,First_Serve%_30matches,First_Serve_Win%_30matches,BPSave%_30matches,BPConv%_30matches,Aces/Match_30matches,DF/Match_30matches,Future Top 30
0,103720,20.752909,1.0,R,AUS,0.70,3857.0,5.898,12096.0,18.495,2354.0,3.599388,0.549180,0.656075,0.691906,0.648855,0.775000,0.750000,103720,Lleyton Hewitt,180.0,30,0.700000,0.689655,0.500000,0.614167,0.723759,0.642105,0.450000,3.633333,2.166667,True
1,103017,22.535250,4.0,R,GER,0.57,2489.0,6.567,6490.0,17.124,1076.0,2.839050,0.445860,0.531646,0.615721,0.456790,0.575758,0.555556,103017,Nicolas Kiefer,183.0,30,0.633333,0.583333,0.571429,0.512586,0.756438,0.631336,0.421053,6.333333,4.800000,True
2,103264,26.904860,9.0,L,ARG,0.49,545.0,3.516,1844.0,11.897,365.0,2.354839,0.250000,0.436508,0.400000,0.559633,0.000000,0.125000,103264,Mariano Puerta,180.0,30,0.333333,0.259259,0.090909,0.655638,0.652213,0.633094,0.423913,3.866667,2.166667,True
3,102845,22.546201,1.0,R,ESP,0.64,2551.0,5.498,7932.0,17.095,1336.0,2.879310,0.500000,0.597297,0.563536,0.704280,0.500000,0.583333,102845,Carlos Moya,190.0,30,0.633333,0.545455,0.454545,0.537364,0.683812,0.648464,0.405229,4.266667,3.666667,True
4,102563,27.214237,7.0,R,SWE,0.54,2881.0,7.981,6187.0,17.139,879.0,2.434903,0.308333,0.489933,0.550505,0.369231,0.666667,0.613636,102563,Thomas Johansson,180.0,30,0.533333,0.428571,0.375000,0.509013,0.754888,0.625571,0.410959,7.666667,3.233333,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,200282,20.711841,18.0,R,AUS,0.62,319.0,4.431,1141.0,15.847,216.0,3.000000,0.333333,0.627119,0.714286,0.222222,0.428571,0.000000,200282,Alex De Minaur,183.0,30,0.466667,0.500000,0.200000,0.611875,0.673690,0.583969,0.401639,4.633333,3.700000,True
242,126774,20.788501,6.0,R,GRE,0.63,790.0,7.980,1813.0,18.313,243.0,2.454545,0.590909,0.593023,0.609375,0.692308,0.555556,0.000000,126774,Stefanos Tsitsipas,193.0,30,0.466667,0.444444,0.454545,0.572467,0.768238,0.600000,0.388889,7.500000,2.466667,True
243,126610,23.581109,8.0,R,ITA,0.54,431.0,7.981,883.0,16.352,115.0,2.129630,0.375000,0.500000,0.409091,0.625000,0.625000,0.000000,126610,Matteo Berrettini,196.0,30,0.500000,0.409091,0.166667,0.606633,0.763902,0.588235,0.345946,7.366667,2.300000,True
244,111815,23.663244,45.0,L,GBR,0.49,298.0,5.418,770.0,14.000,141.0,2.563636,0.214286,0.414634,0.516129,0.500000,0.416667,0.000000,111815,Cameron Norrie,188.0,30,0.466667,0.333333,0.272727,0.627597,0.699960,0.642857,0.392405,5.033333,4.100000,False


In [59]:
### same
PD_merge_predict.shape
columns_drop = ['win_count', 'lose_count', 'match_count', 'aces_in_match_lost', 'aces_in_match_won', 'first_serve_percentage_match_lost',
              'first_serve_percentage_match_won', 'first_serve_won_percentage_match_lost', 'first_serve_won_percentage_match_won', 'breakpoints_won_match_lost',
              'breakpoints_won_match_won', 'win_count_30', 'loss_count_30', 'win_count_100', 'loss_count_100', 'total_top30_matches',
              'total_top100_matches', 'hard_surface_win', 'hard_surface_loss', 'hard_surface_match_count', 
              'clay_surface_win', 'clay_surface_loss', 'clay_surface_match_count',
              'grass_surface_win', 'grass_surface_loss', 'grass_surface_match_count',
              'carpet_surface_win', 'carpet_surface_loss', 'carpet_surface_match_count']

PD_merge_predict = PD_merge_predict.drop(columns_drop, 1)
PD_merge_predict


<ipython-input-59-f1a6e467d9af>:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  PD_merge_predict = PD_merge_predict.drop(columns_drop, 1)


KeyError: "['win_count' 'lose_count' 'match_count' 'aces_in_match_lost'\n 'aces_in_match_won' 'first_serve_percentage_match_lost'\n 'first_serve_percentage_match_won'\n 'first_serve_won_percentage_match_lost'\n 'first_serve_won_percentage_match_won' 'breakpoints_won_match_lost'\n 'breakpoints_won_match_won' 'win_count_30' 'loss_count_30'\n 'win_count_100' 'loss_count_100' 'total_top30_matches'\n 'total_top100_matches' 'hard_surface_win' 'hard_surface_loss'\n 'hard_surface_match_count' 'clay_surface_win' 'clay_surface_loss'\n 'clay_surface_match_count' 'grass_surface_win' 'grass_surface_loss'\n 'grass_surface_match_count' 'carpet_surface_win' 'carpet_surface_loss'\n 'carpet_surface_match_count'] not found in axis"

In [ ]:
PD_merge_30 = pd.read_csv('PD_merge_30.csv')
PD_merge_30
#columns_drop =['Height(cm)', 'lose_count', 'match_count', 'win%', 'Unnamed: 0.1']


In [ ]:
#saving combo to csv
#PD_merge_30.to_csv('PD_merge_30.csv')


In [ ]:
### Future prediction ---
#PD_merge_predict['Top30'] == PD_merge_predict['MaxRank'] <= 30 
#PD_merge_predict[PD_merge_predict['MaxRank'] <= 30] 



In [ ]:
PD_merge[PD_merge.MaxRank == 1]

## NOTES:

- There are only five players in the Top 50 of the ATP Rankings that hit on average at least one ace per service game in 2018. Aces are the ultimate serve weapon, but performance varies greatly at the pinnacle of our sport with the number of aces hit per game

In [ ]:
Ivo Karlovic

Feder, Murray, Nadal, Roddick, Djokovic

#win% likely wrong
aces_in_match_lost
match_wins & match_loss

l_first_serve = first serve percentage matches lost

ace_count / match_count() - check

In [ ]:
# ### BULK FUNCTION TO BUILD OUT PLAYER DATABASE ### -

# PlayerColumns = ['PlayerID', 'Player_Name', 'Age', 'Height', 'MaxRank', 'Hand', 'Country', 'Matches', 'Aces', 'Double_Faults', 'First_Serve_%', 'First_Serve_Win_%', 'Breakpoints_Won']

# PlayerData = pd.DataFrame(columns = PlayerColumns)
# PlayerData.set_index('PlayerID')

# #Creating data for the PlayerData dataframe
# for index, row in df.iterrows():
#     #player was id as player_1
#     p1_id = int(row.player_1_id)
    
#     #checking
#     if (p1_id not in PlayerData.index):
#         PlayerData.loc[p1_id] = [p1_id, row.player_1_name, row.player_1_age, row.player_1_ht, row.player_1_rank, row.player_1_hand, row.player_1_ioc, row.player_1_counter, row.player_1_aces, row.player_1_double_faults, row.player_1_first_serve_percent, row.player_1_first_serve_win_percent, row.player_1_break_points_won]

#     #Updating best rank
#     #Update rank with best rank value
#     if(row.player_1_rank < PlayerData.loc[p1_id].MaxRank and not np.isnan(PlayerData.loc[p1_id].MaxRank)):
#         PlayerData.loc[p1_id] = [p1_id, row.player_1_name, row.player_1_age, row.player_1_ht, row.player_1_rank, row.player_1_hand, row.player_1_ioc, row.player_1_counter ,row.player_1_aces, row.player_1_double_faults, row.player_1_first_serve_percent, row.player_1_first_serve_win_percent, row.player_1_break_points_won]
        
#     else:
#         PlayerData.loc[p1_id] = [p1_id, row.player_1_name, row.player_1_age, row.player_1_ht, row.player_1_rank, row.player_1_hand, row.player_1_ioc, row.player_1_counter, row.player_1_aces, row.player_1_double_faults, row.player_1_first_serve_percent, row.player_1_first_serve_win_percent, row.player_1_break_points_won]
        
        
#     #same code but for player2 (as they were set randomly)
#     p2_id = int(row.player_2_id)
    
#     #checking
#     if (p2_id not in PlayerData.index):
#         PlayerData.loc[p2_id] = [p2_id, row.player_2_name, row.player_2_age, row.player_2_ht, row.player_2_rank, row.player_2_hand, row.player_2_ioc, row.player_2_counter, row.player_2_aces, row.player_2_double_faults, row.player_2_first_serve_percent, row.player_2_first_serve_win_percent, row.player_2_break_points_won]

#     #Updating best rank
#     #Update rank with best rank value
#     if(row.player_2_rank < PlayerData.loc[p2_id].MaxRank and not np.isnan(PlayerData.loc[p2_id].MaxRank)):
#         PlayerData.loc[p2_id] = [p2_id, row.player_2_name, row.player_2_age, row.player_2_ht, row.player_2_rank, row.player_2_hand, row.player_2_ioc, row.player_2_counter, row.player_2_aces, row.player_2_double_faults, row.player_2_first_serve_percent, row.player_2_first_serve_win_percent, row.player_2_break_points_won]
        
#     else:
#         PlayerData.loc[p2_id] = [p2_id, row.player_2_name, row.player_2_age, row.player_2_ht, row.player_2_rank, row.player_2_hand, row.player_2_ioc, row.player_2_counter, row.player_2_aces, row.player_2_double_faults, row.player_2_first_serve_percent, row.player_2_first_serve_win_percent, row.player_2_break_points_won]

In [ ]:
import re
def total_games(s):
    num_list = re.findall('\d+', s)
    num_list = [int(x) for x in num_list]
    return sum(num_list)

df['total_games'] = df['score'].apply(total_games)
df

In [ ]:
# ### creating databases for surface type
# hard = df[df.surface == 'Hard']
# clay = df[df.surface == 'Clay']
# grass = df[df.surface == 'Grass']
# carpet = df[df.surface == 'Carpet']

# #Hard
# hard_win = hard.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(int).reset_index()
# hard_win = hard_win.rename(columns={'winner_name': 'hard_surface_win', 'winner_id' : 'PlayerID'})
# hard_loss = hard.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(int).reset_index()
# hard_loss = hard_loss.rename(columns={'loser_name': 'hard_surface_loss', 'loser_id' : 'PlayerID'})
# data_frames = [hard_win, hard_loss]
# hard_df = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
#                                             how='left'), data_frames)
# #putting losses to 0 count
# hard_df = hard_df.fillna(0)
# hard_df['hard_surface_match_count'] = hard_df['hard_surface_win'] + hard_df['hard_surface_loss']
# hard_df['hard_win%'] = hard_df['hard_surface_win'] / hard_df['hard_surface_match_count']

# # #Clay
# clay_win = clay.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(int).reset_index()
# clay_win = clay_win.rename(columns={'winner_name': 'clay_surface_win', 'winner_id' : 'PlayerID'})
# clay_loss = clay.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(int).reset_index()
# clay_loss = clay_loss.rename(columns={'loser_name': 'clay_surface_loss', 'loser_id' : 'PlayerID'})
# data_frames = [clay_win, clay_loss]
# clay_df = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
#                                             how='left'), data_frames)
# #putting losses to 0 count
# clay_df = clay_df.fillna(0)
# clay_df['clay_surface_match_count'] = clay_df['clay_surface_win'] + clay_df['clay_surface_loss']
# clay_df['clay_win%'] = clay_df['clay_surface_win'] / clay_df['clay_surface_match_count']


# #Grass
# grass_win = grass.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(int).reset_index()
# grass_win = grass_win.rename(columns={'winner_name': 'grass_surface_win', 'winner_id' : 'PlayerID'})
# grass_loss = grass.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(int).reset_index()
# grass_loss = grass_loss.rename(columns={'loser_name': 'grass_surface_loss', 'loser_id' : 'PlayerID'})
# data_frames = [grass_win, grass_loss]
# grass_df = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
#                                             how='left'), data_frames)
# #putting losses to 0 count
# grass_df = grass_df.fillna(0)
# grass_df['grass_surface_match_count'] = grass_df['grass_surface_win'] + grass_df['grass_surface_loss']
# grass_df['grass_win%'] = grass_df['grass_surface_win'] / grass_df['grass_surface_match_count']

# #Carpet
# carpet_win = carpet.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(int).reset_index()
# carpet_win = carpet_win.rename(columns={'winner_name': 'carpet_surface_win', 'winner_id' : 'PlayerID'})
# carpet_loss = carpet.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(int).reset_index()
# carpet_loss = carpet_loss.rename(columns={'loser_name': 'carpet_surface_loss', 'loser_id' : 'PlayerID'})
# data_frames = [carpet_win, carpet_loss]
# carpet_df = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
#                                             how='left'), data_frames)
# #putting losses to 0 count
# carpet_df = carpet_df.fillna(0)
# carpet_df['carpet_surface_match_count'] = carpet_df['carpet_surface_win'] + carpet_df['carpet_surface_loss']
# carpet_df['carpet_win%'] = carpet_df['carpet_surface_win'] / carpet_df['carpet_surface_match_count']

# data_frames = [hard_df, clay_df, grass_df, carpet_df]
# surface = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
#                                             how='left'), data_frames)

# surface.isnull().sum()